In [353]:
import importlib
%load_ext autoreload
%autoreload 2

import pytorch_tabular
from pytorch_tabular.pytorch_tabular import TabularModel
from pytorch_tabular.pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

np.random.seed(0)

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
from imblearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import metrics

import rich

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Death Prediction

In [347]:
X_train = np.load('X_train_static_6.npy')
X_test = np.load('X_test_static_6.npy')
y_train = np.load('y_train_static_6.npy')
y_test = np.load('y_test_static_6.npy')

In [231]:
len(features)

193

In [297]:
features = ['Time Until Discharge',
 'APACHE Score',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'Hour of Admission',
 'Teaching Status',
 'Ethnicity: African American',
 'Ethnicity: Asian',
 'Ethnicity: Caucasian',
 'Ethnicity: Hispanic',
 'Ethnicity: Native American',
 'Ethnicity: Other/Unknown',
 'CCU-CTICU',
 'CSICU',
 'CTICU',
 'Cardiac ICU',
 'MICU',
 'Med-Surg ICU',
 'Neuro ICU',
 'SICU',
 'Admission Source: Acute Care/Floor',
 'Admission Source: Direct Admit',
 'Admission Source: Emergency Department',
 'Admission Source: Floor',
 'Admission Source: Operating Room',
 'Admission Source: Other Hospital',
 'Admission Source: PACU',
 'Admission Source: Recovery Room',
 'Admission Source: Step-Down Unit (SDU)',
 'Admission Source: misc',
 'unitvisitnumber_1',
 'unitvisitnumber_2',
 'unitvisitnumber_3',
 'unitvisitnumber_misc',
 'Admission',
 'Readmission',
 'Transfer',
 'Physician Speciality: Not Specified',
 'Physician Speciality: Cardiology',
 'Physician Speciality: Critical care medicine (CCM)',
 'Physician Speciality: Family practice',
 'Physician Speciality: Hospitalist',
 'Physician Speciality: Internal medicine',
 'Physician Speciality: Misc',
 'Physician Speciality: Neurology',
 'Physician Speciality: Other',
 'Physician Speciality: Pulmonary',
 'Physician Speciality: Pulmonary/CCM',
 'Physician Speciality: Surgery-cardiac',
 'Physician Speciality: Surgery-general',
 'Physician Speciality: Surgery-neuro',
 'Physician Speciality: Surgery-trauma',
 'Physician Speciality: Surgery-vascular',
 'Physician Speciality: Unknown',
 'Number of Beds: 100 - 249',
 'Number of Beds: 250 - 499',
 'Number of Beds: <100',
 'Number of Beds: >= 500',
 'Region: Midwest',
 'Region: Northeast',
 'Region: South',
 'Region: West',
 'Systolic BP_mean',
 'Diastolic BP_mean',
 'Mean BP_mean',
 'Systolic BP_std',
 'Diastolic BP_std',
 'Mean BP_std',
 'Exhaled MV_mean',
 'Exhaled TV (patient)_mean',
 'FiO2_mean',
 'LPM O2_mean',
 'Mean Airway Pressure_mean',
 'PEEP_mean',
 'Peak Insp. Pressure_mean',
 'Plateau Pressure_mean',
 'Pressure Support_mean',
 'RR_mean',
 'SaO2_mean',
 'TV/kg IBW_mean',
 'Tidal Volume_mean',
 'Total RR_mean',
 'Vent Rate_mean',
 'Exhaled MV_std',
 'Exhaled TV_std',
 'FiO2_std',
 'LPM O2_std',
 'Mean Airway Pressure_std',
 'PEEP_std',
 'Peak Insp. Pressure_std',
 'Plateau Pressure_std',
 'Pressure Support_std',
 'RR_std',
 'SaO2_std',
 'TV/kg IBW_std',
 'Tidal Volume_std',
 'Total RR_std',
 'Vent Rate_std',
 'Basos_mean',
 'Eos_mean',
 'Lymphocytes_mean',
 'Monos_mean',
 'Polys_mean',
 'ALT_mean',
 'AST_mean',
 'BUN_mean',
 'Base Excess_mean',
 'FIO2_mean',
 'HCO3_mean',
 'Hct_mean',
 'Hgb_mean',
 'MCH_mean',
 'MCHC_mean',
 'MCV_mean',
 'MPV_mean',
 'O2 Sat_mean',
 'PT_mean',
 'PT - INR_mean',
 'PTT_mean',
 'RBC_mean',
 'RDW_mean',
 'WBC_mean',
 'Albumin_mean',
 'Alkaline phos._mean',
 'Anion gap_mean',
 'Bedside glucose_mean',
 'Bicarbonate_mean',
 'Calcium_mean',
 'Chloride_mean',
 'Creatinine_mean',
 'Glucose_mean',
 'Lactate_mean',
 'Magnesium_mean',
 'pH_mean',
 'paCO2_mean',
 'paO2_mean',
 'Phosphate_mean',
 'Platelets_mean',
 'Potassium_mean',
 'Sodium_mean',
 'Bilirubin_mean',
 'Protein_mean',
 'Troponin_mean',
 'Urinary specific gravity_mean',
 'Basos_std',
 'EOS_std',
 'Lymphocytes_std',
 'Monos_std',
 'Polys_std',
 'ALT_std',
 'AST_std',
 'BUN_std',
 'Base Excess_std',
 'FIO2_std',
 'HCO3_std',
 'Hct_std',
 'Hgb_std',
 'MCH_std',
 'MCHC_std',
 'MCV_std',
 'MPV_std',
 'O2 Sat_std',
 'PT_std',
 'PT - INR_std',
 'PTT_std',
 'RBC_std',
 'RDW_std',
 'WBC_std',
 'Albumin_std',
 'Alkaline phos._std',
 'Anion gap_std',
 'Bedside glucose_std',
 'Bicarbonate_std',
 'Calcium_std',
 'Chloride_std',
 'Creatinine_std',
 'Glucose_std',
 'Lactate_std',
 'Magnesium_std',
 'pH_std',
 'paCO2_std',
 'paO2_std',
 'Phosphate_std',
 'Platelets_std',
 'Potassium_std',
 'Sodium_std',
 'Bilirubin_std',
 'Protein_std',
 'Troponin_std',
 'Urinary specific gravity_std',
           'Death']

In [298]:
from collections import Counter
[k for k,v in Counter(num_col_names).items() if v>1]

[]

In [299]:
num_col_names = ['Time Until Discharge',
 'APACHE Score', 'Age',
 'Height',
 'Weight', 'Hour of Admission', 'Systolic BP_mean',
 'Diastolic BP_mean',
 'Mean BP_mean',
 'Systolic BP_std',
 'Diastolic BP_std',
 'Mean BP_std',
 'Exhaled MV_mean',
 'Exhaled TV (patient)_mean',
 'FiO2_mean',
 'LPM O2_mean',
 'Mean Airway Pressure_mean',
 'PEEP_mean',
 'Peak Insp. Pressure_mean',
 'Plateau Pressure_mean',
 'Pressure Support_mean',
 'RR_mean',
 'SaO2_mean',
 'TV/kg IBW_mean',
 'Tidal Volume_mean',
 'Total RR_mean','Vent Rate_mean',
 'Exhaled MV_std',
 'Exhaled TV_std',
 'FiO2_std',
 'LPM O2_std',
 'Mean Airway Pressure_std',
 'PEEP_std',
 'Peak Insp. Pressure_std',
 'Plateau Pressure_std',
 'Pressure Support_std',
 'RR_std',
 'SaO2_std',
 'TV/kg IBW_std',
 'Tidal Volume_std',
 'Total RR_std',
 'Vent Rate_std',
 'Basos_mean',
 'Eos_mean',
 'Lymphocytes_mean',
 'Monos_mean',
 'Polys_mean',
 'ALT_mean',
 'AST_mean',
 'BUN_mean',
 'Base Excess_mean',
 'FIO2_mean',
 'HCO3_mean',
 'Hct_mean',
 'Hgb_mean',
 'MCH_mean',
 'MCHC_mean',
 'MCV_mean',
 'MPV_mean',
 'O2 Sat_mean',
 'PT_mean',
 'PT - INR_mean',
 'PTT_mean',
 'RBC_mean',
 'RDW_mean',
 'WBC_mean','Albumin_mean',
 'Alkaline phos._mean',
 'Anion gap_mean',
 'Bedside glucose_mean',
 'Bicarbonate_mean',
 'Calcium_mean',
 'Chloride_mean',
 'Creatinine_mean',
 'Glucose_mean',
 'Lactate_mean',
 'Magnesium_mean',
 'pH_mean',
 'paCO2_mean',
 'paO2_mean',
 'Phosphate_mean',
 'Platelets_mean',
 'Potassium_mean',
 'Sodium_mean',
 'Bilirubin_mean',
 'Protein_mean',
 'Troponin_mean',
 'Urinary specific gravity_mean',
 'Basos_std',
 'EOS_std',
 'Lymphocytes_std',
 'Monos_std',
 'Polys_std',
 'ALT_std',
 'AST_std',
 'BUN_std', 'Base Excess_std',
 'FIO2_std',
 'HCO3_std',
 'Hct_std',
 'Hgb_std',
 'MCH_std',
 'MCHC_std',
 'MCV_std',
 'MPV_std',
 'O2 Sat_std',
 'PT_std',
 'PT - INR_std',
 'PTT_std',
 'RBC_std',
 'RDW_std',
 'WBC_std',
 'Albumin_std',
 'Alkaline phos._std',
 'Anion gap_std',
 'Bedside glucose_std',
 'Bicarbonate_std',
 'Calcium_std',
 'Chloride_std',
 'Creatinine_std',
 'Glucose_std',
 'Lactate_std',
 'Magnesium_std',
 'pH_std',
 'paCO2_std',
 'paO2_std',
 'Phosphate_std',
 'Platelets_std',
 'Potassium_std',
 'Sodium_std',
 'Bilirubin_std',
 'Protein_std',
 'Troponin_std',
 'Urinary specific gravity_std']

In [300]:
cat_col_names = ['Sex', 'Teaching Status',
 'Ethnicity: African American',
 'Ethnicity: Asian',
 'Ethnicity: Caucasian',
 'Ethnicity: Hispanic',
 'Ethnicity: Native American',
 'Ethnicity: Other/Unknown',
 'CCU-CTICU',
 'CSICU',
 'CTICU',
 'Cardiac ICU',
 'MICU',
 'Med-Surg ICU',
 'Neuro ICU',
 'SICU',
 'Admission Source: Acute Care/Floor',
 'Admission Source: Direct Admit',
 'Admission Source: Emergency Department',
 'Admission Source: Floor',
 'Admission Source: Operating Room',
 'Admission Source: Other Hospital',
 'Admission Source: PACU',
 'Admission Source: Recovery Room',
 'Admission Source: Step-Down Unit (SDU)',
 'Admission Source: misc',
 'unitvisitnumber_1',
 'unitvisitnumber_2',
 'unitvisitnumber_3',
 'unitvisitnumber_misc',
 'Admission',
 'Readmission',
 'Transfer',
 'Physician Speciality: Not Specified',
 'Physician Speciality: Cardiology',
 'Physician Speciality: Critical care medicine (CCM)',
 'Physician Speciality: Family practice',
 'Physician Speciality: Hospitalist',
 'Physician Speciality: Internal medicine',
 'Physician Speciality: Misc',
 'Physician Speciality: Neurology',
 'Physician Speciality: Other',
 'Physician Speciality: Pulmonary',
 'Physician Speciality: Pulmonary/CCM',
 'Physician Speciality: Surgery-cardiac',
 'Physician Speciality: Surgery-general',
 'Physician Speciality: Surgery-neuro',
 'Physician Speciality: Surgery-trauma',
 'Physician Speciality: Surgery-vascular',
 'Physician Speciality: Unknown',
 'Number of Beds: 100 - 249',
 'Number of Beds: 250 - 499',
 'Number of Beds: <100',
 'Number of Beds: >= 500',
 'Region: Midwest',
 'Region: Northeast',
 'Region: South',
 'Region: West']

In [301]:
set(list(set(features)-set(num_col_names))) - set(cat_col_names)

{'Death'}

In [303]:
# Get the validation folds
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=33)

In [340]:
# Generate the parameter grid.
param_grid = dict(batch_size = [32, 64, 128],
                  num_layers = [1, 2, 3],
                  num_trees = [32, 64, 128, 256],
                  depth = [6, 8]
)

grid = ParameterGrid(param_grid)

optimizer_config = OptimizerConfig()

search_results = pd.DataFrame() 
for params in grid:
    scores = []
    for train_ix, test_ix in cv.split(X_train, y_train):
        # split data
        X_train_2, X_test_2 = X_train[train_ix, :], X_train[test_ix, :]
        y_train_2, y_test_2 = y_train[train_ix], y_train[test_ix]
        
        imp_mean_2 = IterativeImputer(random_state=0)
        imp_mean_2.fit(X_train_2)
        X_train_2 = imp_mean_2.transform(X_train_2)
        X_test_2 = imp_mean_2.transform(X_test_2)
        
        y_train_2 = y_train_2.reshape(-1,1)
        y_test_2 = y_test_2.reshape(-1,1)

        X_train_2 = np.append(X_train_2, y_train_2, axis=1)
        X_test_2 = np.append(X_test_2, y_test_2, axis=1)
        
        df_train = pd.DataFrame(X_train_2, columns = features)
        df_test = pd.DataFrame(X_test_2, columns = features)
        
        data_config = DataConfig(
        target=['Death'],
        continuous_cols=num_col_names,
        categorical_cols=cat_col_names,
        normalize_continuous_features=False,
        validation_split=0.2
    )
        
        trainer_config = TrainerConfig(
        auto_lr_find=True,
        batch_size=params['batch_size'],
        max_epochs=100,
        auto_select_gpus=False,
        gpus=0,
        # track_grad_norm=2,
        gradient_clip_val=10,
        progress_bar='simple'
    )
        
        model_config = NodeConfig(
        task="classification",
        num_layers=params['num_layers'],
        num_trees=params['num_trees'],
        learning_rate=1e-3,
        depth=params['depth'],
        embed_categorical=False,
    )
        
        tabular_model = TabularModel(
        data_config=data_config,
        model_config=model_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
    )
        
        tabular_model.fit(train=df_train, test=df_test)
        
        y_prob = tabular_model.predict(df_test)['1.0_probability']
        
        score = roc_auc_score(y_test_2, y_prob)
        scores.append(score)
    print('For: ', params)
    print('Score is: ', np.mean(scores))
    results = pd.DataFrame([params])
    results['score'] = np.round(np.mean(scores), 3)
    results['std'] = np.round(np.std(scores), 3)
    search_results = search_results.append(results)

Global seed set to 42
/Users/mmesinovic/Desktop/AMI/pytorch_tabular/pytorch_tabular/models/node/odst.py:105: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, u

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Restoring states from the checkpoint path at /Users/mmesinovic/Desktop/AMI/lr_find_temp_model_67419349-1338-4e88-a123-70d8e2a556b2.ckpt
Learning rate set to 0.2290867652767775

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | NODEBackbone     | 94.7 K
1 | head     | Lambda           | 0     
2 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
90.6 K    Trainable params
4.1 K     Non-trainable params
94.7 K    Total params
0.379     Total estimated model params size (MB)
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /Users/mmesinovic/Desktop/AMI/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Generating Predictions...:   0%|          | 0/12 [00:00<?, ?it/s]

For:  {'batch_size': 128, 'depth': 8, 'num_layers': 3, 'num_trees': 256}
Score is:  0.8655777696310037


/var/folders/rb/zrr_wqjs59v2n19lg6kssgnw0000gr/T/ipykernel_62762/1665362633.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  search_results = search_results.append(results)


In [341]:
search_results

,batch_size,depth,num_layers,num_trees,score,std
0,128,8,1,32,0.850,0.020
0,128,8,1,64,0.846,0.031
0,128,8,1,128,0.867,0.012
0,128,8,1,256,0.870,0.010
0,128,8,2,32,0.864,0.005
0,128,8,2,64,0.863,0.015
0,128,8,2,128,0.868,0.013
0,128,8,2,256,0.869,0.011
0,128,8,3,32,0.856,0.023
0,128,8,3,64,0.865,0.017


In [339]:
search_results

,batch_size,depth,num_layers,num_trees,score,std
0,64,6,1,32,0.858,0.016
0,64,6,1,64,0.857,0.019
0,64,6,1,128,0.864,0.016
0,64,6,1,256,0.877,0.009
0,64,6,2,32,0.869,0.008
0,64,6,2,64,0.874,0.008
0,64,6,2,128,0.873,0.005
0,64,6,2,256,0.879,0.007
0,64,6,3,32,0.868,0.010
0,64,6,3,64,0.877,0.009


In [337]:
search_results

,batch_size,depth,num_layers,num_trees,score,std
0,32,6,1,32,0.848,0.016
0,32,6,1,64,0.850,0.011
0,32,6,1,128,0.863,0.012
0,32,6,1,256,0.866,0.011
0,32,6,2,32,0.853,0.012
0,32,6,2,64,0.859,0.019
0,32,6,2,128,0.869,0.006
0,32,6,2,256,0.868,0.012


0	64	6	2	256	0.879	0.007

In [306]:
y_train_2 = y_train.reshape(-1,1)
y_test_2 = y_test.reshape(-1,1)

X_train_2 = np.append(X_train, y_train_2, axis=1)
X_test_2 = np.append(X_test, y_test_2, axis=1)

df_train = pd.DataFrame(X_train_2, columns = features)
df_test = pd.DataFrame(X_test_2, columns = features)

In [349]:
imp_mean_2 = IterativeImputer(random_state=0)
imp_mean_2.fit(X_train)
X_train_2 = imp_mean_2.transform(X_train)
X_test_2 = imp_mean_2.transform(X_test)

y_train_2 = y_train.reshape(-1,1)
y_test_2 = y_test.reshape(-1,1)

X_train_2 = np.append(X_train_2, y_train_2, axis=1)
X_test_2 = np.append(X_test_2, y_test_2, axis=1)

df_train = pd.DataFrame(X_train_2, columns = features)
df_test = pd.DataFrame(X_test_2, columns = features)

data_config = DataConfig(
target=['Death'],
continuous_cols=num_col_names,
categorical_cols=cat_col_names,
normalize_continuous_features=False,
validation_split=0.2
)

trainer_config = TrainerConfig(
auto_lr_find=True,
batch_size=64,
max_epochs=100,
auto_select_gpus=False,
gpus=0,
# track_grad_norm=2,
gradient_clip_val=10,
progress_bar='simple'
)

model_config = NodeConfig(
task="classification",
num_layers=2,
num_trees=256,
learning_rate=1e-3,
depth=6,
embed_categorical=False,
)

tabular_model = TabularModel(
data_config=data_config,
model_config=model_config,
optimizer_config=optimizer_config,
trainer_config=trainer_config,
)

In [350]:
tabular_model.fit(train=df_train, test=df_test)

Global seed set to 42
/Users/mmesinovic/Desktop/AMI/pytorch_tabular/pytorch_tabular/models/node/odst.py:105: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, u

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Restoring states from the checkpoint path at /Users/mmesinovic/Desktop/AMI/lr_find_temp_model_5a244071-9f33-46db-992d-8365045cdfa4.ckpt
Learning rate set to 0.13182567385564073

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | NODEBackbone     | 2.7 M 
1 | head     | Lambda           | 0     
2 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
2.7 M     Trainable params
1.5 K     Non-trainable params
2.7 M     Total params
10.910    Total estimated model params size (MB)
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /Users/mmesinovic/Desktop/AMI/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [351]:
y_prob = tabular_model.predict(df_test)['1.0_probability']
y_pred = tabular_model.predict(df_test)['prediction']

Generating Predictions...:   0%|          | 0/29 [00:00<?, ?it/s]

Generating Predictions...:   0%|          | 0/29 [00:00<?, ?it/s]

In [354]:
print('Results:')
print('AUROC is:', metrics.roc_auc_score(y_test_2, y_prob))
print('Accuracy is:', metrics.accuracy_score(y_test_2, y_pred))
print('Average Precision is:', metrics.average_precision_score(y_test_2, y_prob))
print('Weighted F1 is:', metrics.f1_score(y_test_2, y_pred, average='weighted'))
print('Sensitivity is:', metrics.recall_score(y_test_2, y_pred))
print('#################################################')

Results:
AUROC is: 0.8628845391627793
Accuracy is: 0.8609913793103449
Average Precision is: 0.6634577310196135
Weighted F1 is: 0.850864015895938
Sensitivity is: 0.47701149425287354
#################################################


# MI Prediction

In [355]:
X_train = np.load('X_train_static_MI.npy')
X_test = np.load('X_test_static_MI.npy')
y_train = np.load('y_train_static_MI.npy')
y_test = np.load('y_test_static_MI.npy')

In [370]:
features = ['Time Until Discharge',
 'APACHE Score',
 'Sex',
 'Age',
 'Height',
 'Weight',
 'Hour of Admission',
 'Teaching Status',
 'Ethnicity: African American',
 'Ethnicity: Asian',
 'Ethnicity: Caucasian',
 'Ethnicity: Hispanic',
 'Ethnicity: Native American',
 'Ethnicity: Other/Unknown',
 'CCU-CTICU',
 'CSICU',
 'CTICU',
 'Cardiac ICU',
 'MICU',
 'Med-Surg ICU',
 'Neuro ICU',
 'SICU',
 'Admission Source: Acute Care/Floor',
 'Admission Source: Direct Admit',
 'Admission Source: Emergency Department',
 'Admission Source: Floor',
 'Admission Source: Operating Room',
 'Admission Source: Other Hospital',
 'Admission Source: PACU',
 'Admission Source: Recovery Room',
 'Admission Source: Step-Down Unit (SDU)',
 'Admission Source: misc',
 'unitvisitnumber_1',
 'unitvisitnumber_2',
 'unitvisitnumber_3',
 'unitvisitnumber_misc',
 'Admission',
 'Readmission',
 'Transfer',
 'Physician Speciality: Not Specified',
 'Physician Speciality: Cardiology',
 'Physician Speciality: Critical care medicine (CCM)',
 'Physician Speciality: Family practice',
 'Physician Speciality: Hospitalist',
 'Physician Speciality: Internal medicine',
 'Physician Speciality: Misc',
 'Physician Speciality: Neurology',
 'Physician Speciality: Other',
 'Physician Speciality: Pulmonary',
 'Physician Speciality: Pulmonary/CCM',
 'Physician Speciality: Surgery-cardiac',
 'Physician Speciality: Surgery-general',
 'Physician Speciality: Surgery-neuro',
 'Physician Speciality: Surgery-trauma',
 'Physician Speciality: Surgery-vascular',
 'Physician Speciality: Unknown',
 'Number of Beds: 100 - 249',
 'Number of Beds: 250 - 499',
 'Number of Beds: <100',
 'Number of Beds: >= 500',
 'Region: Midwest',
 'Region: Northeast',
 'Region: South',
 'Region: West',
 'Systolic BP_mean',
 'Diastolic BP_mean',
 'Mean BP_mean',
 'Systolic BP_std',
 'Diastolic BP_std',
 'Mean BP_std',
 'Exhaled MV_mean',
 'Exhaled TV (patient)_mean',
 'FiO2_mean',
 'LPM O2_mean',
 'Mean Airway Pressure_mean',
 'PEEP_mean',
 'Peak Insp. Pressure_mean',
 'Plateau Pressure_mean',
 'Pressure Support_mean',
 'RR_mean',
 'SaO2_mean',
 'TV/kg IBW_mean',
 'Tidal Volume_mean',
 'Total RR_mean',
 'Vent Rate_mean',
 'Exhaled MV_std',
 'Exhaled TV_std',
 'FiO2_std',
 'LPM O2_std',
 'Mean Airway Pressure_std',
 'PEEP_std',
 'Peak Insp. Pressure_std',
 'Plateau Pressure_std',
 'Pressure Support_std',
 'RR_std',
 'SaO2_std',
 'TV/kg IBW_std',
 'Tidal Volume_std',
 'Total RR_std',
 'Vent Rate_std',
 'Basos_mean',
 'Eos_mean',
 'Lymphocytes_mean',
 'Monos_mean',
 'Polys_mean',
 'ALT_mean',
 'AST_mean',
 'BUN_mean',
 'Base Excess_mean',
 'FIO2_mean',
 'HCO3_mean',
 'Hct_mean',
 'Hgb_mean',
 'MCH_mean',
 'MCHC_mean',
 'MCV_mean',
 'MPV_mean',
 'O2 Sat_mean',
 'PT_mean',
 'PT - INR_mean',
 'PTT_mean',
 'RBC_mean',
 'RDW_mean',
 'WBC_mean',
 'Albumin_mean',
 'Alkaline phos._mean',
 'Anion gap_mean',
 'Bedside glucose_mean',
 'Bicarbonate_mean',
 'Calcium_mean',
 'Chloride_mean',
 'Creatinine_mean',
 'Glucose_mean',
 'Lactate_mean',
 'Magnesium_mean',
 'pH_mean',
 'paCO2_mean',
 'paO2_mean',
 'Phosphate_mean',
 'Platelets_mean',
 'Potassium_mean',
 'Sodium_mean',
 'Bilirubin_mean',
 'Protein_mean',
 'Troponin_mean',
 'Urinary specific gravity_mean',
 'Basos_std',
 'EOS_std',
 'Lymphocytes_std',
 'Monos_std',
 'Polys_std',
 'ALT_std',
 'AST_std',
 'BUN_std',
 'Base Excess_std',
 'FIO2_std',
 'HCO3_std',
 'Hct_std',
 'Hgb_std',
 'MCH_std',
 'MCHC_std',
 'MCV_std',
 'MPV_std',
 'O2 Sat_std',
 'PT_std',
 'PT - INR_std',
 'PTT_std',
 'RBC_std',
 'RDW_std',
 'WBC_std',
 'Albumin_std',
 'Alkaline phos._std',
 'Anion gap_std',
 'Bedside glucose_std',
 'Bicarbonate_std',
 'Calcium_std',
 'Chloride_std',
 'Creatinine_std',
 'Glucose_std',
 'Lactate_std',
 'Magnesium_std',
 'pH_std',
 'paCO2_std',
 'paO2_std',
 'Phosphate_std',
 'Platelets_std',
 'Potassium_std',
 'Sodium_std',
 'Bilirubin_std',
 'Protein_std',
 'Troponin_std',
 'Urinary specific gravity_std', 'diagnosisoffset', 'MI']

In [371]:
num_col_names = ['Time Until Discharge',
 'APACHE Score', 'Age',
 'Height',
 'Weight', 'Hour of Admission', 'Systolic BP_mean',
 'Diastolic BP_mean',
 'Mean BP_mean',
 'Systolic BP_std',
 'Diastolic BP_std',
 'Mean BP_std',
 'Exhaled MV_mean',
 'Exhaled TV (patient)_mean',
 'FiO2_mean',
 'LPM O2_mean',
 'Mean Airway Pressure_mean',
 'PEEP_mean',
 'Peak Insp. Pressure_mean',
 'Plateau Pressure_mean',
 'Pressure Support_mean',
 'RR_mean',
 'SaO2_mean',
 'TV/kg IBW_mean',
 'Tidal Volume_mean',
 'Total RR_mean','Vent Rate_mean',
 'Exhaled MV_std',
 'Exhaled TV_std',
 'FiO2_std',
 'LPM O2_std',
 'Mean Airway Pressure_std',
 'PEEP_std',
 'Peak Insp. Pressure_std',
 'Plateau Pressure_std',
 'Pressure Support_std',
 'RR_std',
 'SaO2_std',
 'TV/kg IBW_std',
 'Tidal Volume_std',
 'Total RR_std',
 'Vent Rate_std',
 'Basos_mean',
 'Eos_mean',
 'Lymphocytes_mean',
 'Monos_mean',
 'Polys_mean',
 'ALT_mean',
 'AST_mean',
 'BUN_mean',
 'Base Excess_mean',
 'FIO2_mean',
 'HCO3_mean',
 'Hct_mean',
 'Hgb_mean',
 'MCH_mean',
 'MCHC_mean',
 'MCV_mean',
 'MPV_mean',
 'O2 Sat_mean',
 'PT_mean',
 'PT - INR_mean',
 'PTT_mean',
 'RBC_mean',
 'RDW_mean',
 'WBC_mean','Albumin_mean',
 'Alkaline phos._mean',
 'Anion gap_mean',
 'Bedside glucose_mean',
 'Bicarbonate_mean',
 'Calcium_mean',
 'Chloride_mean',
 'Creatinine_mean',
 'Glucose_mean',
 'Lactate_mean',
 'Magnesium_mean',
 'pH_mean',
 'paCO2_mean',
 'paO2_mean',
 'Phosphate_mean',
 'Platelets_mean',
 'Potassium_mean',
 'Sodium_mean',
 'Bilirubin_mean',
 'Protein_mean',
 'Troponin_mean',
 'Urinary specific gravity_mean',
 'Basos_std',
 'EOS_std',
 'Lymphocytes_std',
 'Monos_std',
 'Polys_std',
 'ALT_std',
 'AST_std',
 'BUN_std', 'Base Excess_std',
 'FIO2_std',
 'HCO3_std',
 'Hct_std',
 'Hgb_std',
 'MCH_std',
 'MCHC_std',
 'MCV_std',
 'MPV_std',
 'O2 Sat_std',
 'PT_std',
 'PT - INR_std',
 'PTT_std',
 'RBC_std',
 'RDW_std',
 'WBC_std',
 'Albumin_std',
 'Alkaline phos._std',
 'Anion gap_std',
 'Bedside glucose_std',
 'Bicarbonate_std',
 'Calcium_std',
 'Chloride_std',
 'Creatinine_std',
 'Glucose_std',
 'Lactate_std',
 'Magnesium_std',
 'pH_std',
 'paCO2_std',
 'paO2_std',
 'Phosphate_std',
 'Platelets_std',
 'Potassium_std',
 'Sodium_std',
 'Bilirubin_std',
 'Protein_std',
 'Troponin_std',
 'Urinary specific gravity_std']

In [372]:
cat_col_names = ['Sex', 'Teaching Status',
 'Ethnicity: African American',
 'Ethnicity: Asian',
 'Ethnicity: Caucasian',
 'Ethnicity: Hispanic',
 'Ethnicity: Native American',
 'Ethnicity: Other/Unknown',
 'CCU-CTICU',
 'CSICU',
 'CTICU',
 'Cardiac ICU',
 'MICU',
 'Med-Surg ICU',
 'Neuro ICU',
 'SICU',
 'Admission Source: Acute Care/Floor',
 'Admission Source: Direct Admit',
 'Admission Source: Emergency Department',
 'Admission Source: Floor',
 'Admission Source: Operating Room',
 'Admission Source: Other Hospital',
 'Admission Source: PACU',
 'Admission Source: Recovery Room',
 'Admission Source: Step-Down Unit (SDU)',
 'Admission Source: misc',
 'unitvisitnumber_1',
 'unitvisitnumber_2',
 'unitvisitnumber_3',
 'unitvisitnumber_misc',
 'Admission',
 'Readmission',
 'Transfer',
 'Physician Speciality: Not Specified',
 'Physician Speciality: Cardiology',
 'Physician Speciality: Critical care medicine (CCM)',
 'Physician Speciality: Family practice',
 'Physician Speciality: Hospitalist',
 'Physician Speciality: Internal medicine',
 'Physician Speciality: Misc',
 'Physician Speciality: Neurology',
 'Physician Speciality: Other',
 'Physician Speciality: Pulmonary',
 'Physician Speciality: Pulmonary/CCM',
 'Physician Speciality: Surgery-cardiac',
 'Physician Speciality: Surgery-general',
 'Physician Speciality: Surgery-neuro',
 'Physician Speciality: Surgery-trauma',
 'Physician Speciality: Surgery-vascular',
 'Physician Speciality: Unknown',
 'Number of Beds: 100 - 249',
 'Number of Beds: 250 - 499',
 'Number of Beds: <100',
 'Number of Beds: >= 500',
 'Region: Midwest',
 'Region: Northeast',
 'Region: South',
 'Region: West']

In [362]:
from collections import Counter
[k for k,v in Counter(features).items() if v>1]

[]

In [363]:
set(list(set(features)-set(num_col_names))) - set(cat_col_names)

{'MI'}

In [373]:
param_grid = dict(batch_size = [32, 64, 128],
                  num_layers = [1, 2, 3],
                  num_trees = [32, 64, 128, 256],
                  depth = [6, 8]
)

grid = ParameterGrid(param_grid)

optimizer_config = OptimizerConfig()

search_results = pd.DataFrame() 
for params in grid:
    scores = []
    for train_ix, test_ix in cv.split(X_train, y_train):
        # split data
        X_train_2, X_test_2 = X_train[train_ix, :], X_train[test_ix, :]
        y_train_2, y_test_2 = y_train[train_ix], y_train[test_ix]
        
        imp_mean_2 = IterativeImputer(random_state=0)
        imp_mean_2.fit(X_train_2)
        X_train_2 = imp_mean_2.transform(X_train_2)
        X_test_2 = imp_mean_2.transform(X_test_2)
        
        y_train_2 = y_train_2.reshape(-1,1)
        y_test_2 = y_test_2.reshape(-1,1)

        X_train_2 = np.append(X_train_2, y_train_2, axis=1)
        X_test_2 = np.append(X_test_2, y_test_2, axis=1)
        
        df_train = pd.DataFrame(X_train_2, columns = features)
        df_test = pd.DataFrame(X_test_2, columns = features)
        
        data_config = DataConfig(
        target=['MI'],
        continuous_cols=num_col_names,
        categorical_cols=cat_col_names,
        normalize_continuous_features=False,
        validation_split=0.2
    )
        
        trainer_config = TrainerConfig(
        auto_lr_find=True,
        batch_size=params['batch_size'],
        max_epochs=100,
        auto_select_gpus=False,
        gpus=0,
        # track_grad_norm=2,
        gradient_clip_val=10,
        progress_bar='simple'
    )
        
        model_config = NodeConfig(
        task="classification",
        num_layers=params['num_layers'],
        num_trees=params['num_trees'],
        learning_rate=1e-3,
        depth=params['depth'],
        embed_categorical=False,
    )
        
        tabular_model = TabularModel(
        data_config=data_config,
        model_config=model_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
    )
        
        tabular_model.fit(train=df_train, test=df_test)
        
        y_prob = tabular_model.predict(df_test)['1.0_probability']
        
        score = roc_auc_score(y_test_2, y_prob)
        scores.append(score)
    print('For: ', params)
    print('Score is: ', np.mean(scores))
    results = pd.DataFrame([params])
    results['score'] = np.round(np.mean(scores), 3)
    results['std'] = np.round(np.std(scores), 3)
    search_results = search_results.append(results)

Global seed set to 42
/Users/mmesinovic/Desktop/AMI/pytorch_tabular/pytorch_tabular/models/node/odst.py:105: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, u

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Restoring states from the checkpoint path at /Users/mmesinovic/Desktop/AMI/lr_find_temp_model_96e825e0-2eb7-42fc-85b4-1e9ca49afad6.ckpt
Learning rate set to 0.07585775750291836

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | NODEBackbone     | 48.3 K
1 | head     | Lambda           | 0     
2 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
47.5 K    Trainable params
769       Non-trainable params
48.3 K    Total params
0.193     Total estimated model params size (MB)
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /Users/mmesinovic/Desktop/AMI/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Generating Predictions...:   0%|          | 0/12 [00:00<?, ?it/s]

For:  {'batch_size': 128, 'depth': 8, 'num_layers': 3, 'num_trees': 256}
Score is:  0.7873773872518802


/var/folders/rb/zrr_wqjs59v2n19lg6kssgnw0000gr/T/ipykernel_62762/3644184496.py:79: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  search_results = search_results.append(results)


In [374]:
search_results

,batch_size,depth,num_layers,num_trees,score,std
0,32,6,1,32,0.720,0.073
0,32,6,1,64,0.727,0.070
0,32,6,1,128,0.719,0.096
0,32,6,1,256,0.744,0.078
0,32,6,2,32,0.731,0.074
...,...,...,...,...,...,...
0,128,8,2,256,0.785,0.025
0,128,8,3,32,0.756,0.029
0,128,8,3,64,0.763,0.044
0,128,8,3,128,0.773,0.027


In [375]:
search_results.iloc[search_results['score'].argmax()]

batch_size     32.000
depth           8.000
num_layers      3.000
num_trees     128.000
score           0.803
std             0.032
Name: 0, dtype: float64

batch_size     32.000
depth           8.000
num_layers      3.000
num_trees     128.000
score           0.803
std             0.032

In [376]:
imp_mean_2 = IterativeImputer(random_state=0)
imp_mean_2.fit(X_train)
X_train_2 = imp_mean_2.transform(X_train)
X_test_2 = imp_mean_2.transform(X_test)

y_train_2 = y_train.reshape(-1,1)
y_test_2 = y_test.reshape(-1,1)

X_train_2 = np.append(X_train_2, y_train_2, axis=1)
X_test_2 = np.append(X_test_2, y_test_2, axis=1)

df_train = pd.DataFrame(X_train_2, columns = features)
df_test = pd.DataFrame(X_test_2, columns = features)

data_config = DataConfig(
target=['MI'],
continuous_cols=num_col_names,
categorical_cols=cat_col_names,
normalize_continuous_features=False,
validation_split=0.2
)

trainer_config = TrainerConfig(
auto_lr_find=True,
batch_size=32,
max_epochs=100,
auto_select_gpus=False,
gpus=0,
# track_grad_norm=2,
gradient_clip_val=10,
progress_bar='simple'
)

model_config = NodeConfig(
task="classification",
num_layers=3,
num_trees=128,
learning_rate=1e-3,
depth=8,
embed_categorical=False,
)

tabular_model = TabularModel(
data_config=data_config,
model_config=model_config,
optimizer_config=optimizer_config,
trainer_config=trainer_config,
)

/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [377]:
tabular_model.fit(train=df_train, test=df_test)

Global seed set to 42
/Users/mmesinovic/Desktop/AMI/pytorch_tabular/pytorch_tabular/models/node/odst.py:105: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=True)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=True)`.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, u

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Restoring states from the checkpoint path at /Users/mmesinovic/Desktop/AMI/lr_find_temp_model_ba818708-689e-4b9e-b33c-4c3e0221e2c9.ckpt
Learning rate set to 0.15848931924611143

  | Name     | Type             | Params
----------------------------------------------
0 | backbone | NODEBackbone     | 3.1 M 
1 | head     | Lambda           | 0     
2 | loss     | CrossEntropyLoss | 0     
----------------------------------------------
3.1 M     Trainable params
12.3 K    Non-trainable params
3.1 M     Total params
12.263    Total estimated model params size (MB)
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /Users/mmesinovic/Desktop/AMI/saved_models exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/opt/homebrew/Caskroom/miniforge/base/envs/COVID/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [379]:
y_prob = tabular_model.predict(df_test)['1.0_probability']
y_pred = tabular_model.predict(df_test)['prediction']

Generating Predictions...:   0%|          | 0/56 [00:00<?, ?it/s]

Generating Predictions...:   0%|          | 0/56 [00:00<?, ?it/s]

In [380]:
print('Results:')
print('AUROC is:', metrics.roc_auc_score(y_test_2, y_prob))
print('Accuracy is:', metrics.accuracy_score(y_test_2, y_pred))
print('Average Precision is:', metrics.average_precision_score(y_test_2, y_prob))
print('Weighted F1 is:', metrics.f1_score(y_test_2, y_pred, average='weighted'))
print('Sensitivity is:', metrics.recall_score(y_test_2, y_pred))
print('#################################################')

Results:
AUROC is: 0.7496205786966658
Accuracy is: 0.6811023622047244
Average Precision is: 0.7691724517052695
Weighted F1 is: 0.6791467259582489
Sensitivity is: 0.7521739130434782
#################################################
